In [1]:
import pandas as pd 
import numpy as np 
import scipy.stats as stats 

In [79]:
df1 = pd.read_csv('train.csv')
print(df1.shape)

(25383, 13)


In [73]:
df2 = pd.read_csv('test.csv')
print(df2.shape)

(16922, 12)


In [80]:
df1.columns.unique()

Index(['ID', 'danceability', 'energy', 'key', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration', 'genre'],
      dtype='object')

In [81]:
X = df1.drop(["ID", "genre"], axis = 1)
Y = df1['genre']

In [82]:
X

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration
0,0.3341,0.9231,10,0.8540,0.3797,0.0080,0.0002,0.7697,0.7902,0.5391,175
1,0.6344,0.5955,10,0.2826,0.1378,0.0000,0.7137,0.1597,0.4162,0.3043,513
2,0.3983,0.1107,8,0.2459,0.0247,0.7740,0.0000,0.5105,0.4291,0.2324,151
3,0.3139,0.9702,2,0.9252,0.9590,0.0371,0.3015,0.2539,0.0577,0.5406,246
4,0.0735,0.9452,5,0.7702,0.1397,0.2583,0.0015,0.1774,0.1128,0.3176,229
...,...,...,...,...,...,...,...,...,...,...,...
25378,0.8814,0.3398,6,0.4602,0.3825,0.4505,0.0000,0.4222,0.8753,0.4081,88
25379,0.7505,0.5657,0,0.6682,0.1038,0.1281,0.0000,0.0596,0.9583,0.4273,123
25380,0.2520,0.7617,8,0.4928,0.0410,0.0003,0.8395,0.0843,0.1372,0.7828,353
25381,0.6202,0.3455,0,0.6499,0.7960,0.4818,0.0000,0.2007,0.0194,0.9756,120


In [19]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks

In [42]:
#smoteto = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
#X_smt, Y_smt = smoteto.fit_resample(X, Y)

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute   import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [139]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [122]:
pipe_list = [('impute', SimpleImputer()), 
             ('scaler', StandardScaler()),
            ('model', RandomForestClassifier())]
pipe_model = Pipeline(pipe_list)
pipe_model

Pipeline(steps=[('impute', SimpleImputer()), ('scaler', StandardScaler()),
                ('model', RandomForestClassifier())])

In [140]:
hypeer_parameter = {'model__class_weight':['balanced']}
grid_model = GridSearchCV(pipe_model, param_grid=hypeer_parameter, cv=7, 
                         n_jobs = -1 ,scoring='f1')
grid_model.fit(X_train, Y_train)

c:\Users\USER\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the test scores are non-finite: [nan]
  warnings.warn(


GridSearchCV(cv=7,
             estimator=Pipeline(steps=[('impute', SimpleImputer()),
                                       ('scaler', StandardScaler()),
                                       ('model', RandomForestClassifier())]),
             n_jobs=-1, param_grid={'model__class_weight': ['balanced']},
             scoring='f1')

In [141]:
best_model = grid_model.best_estimator_
best_model

Pipeline(steps=[('impute', SimpleImputer()), ('scaler', StandardScaler()),
                ('model', RandomForestClassifier(class_weight='balanced'))])

In [142]:
Y_train_pred = best_model.predict(X_train)
Y_test_pred  = best_model.predict(X_test)

In [143]:
f1_score(Y_train, Y_train_pred,average = "macro")

0.9657880638991292

In [144]:
f1_score(Y_test, Y_test_pred,average = "macro")

0.6344515620270144

In [145]:
X1 = df2.drop(["ID"], axis = 1)


In [146]:
X1_test_pred = best_model.predict(X1)

In [147]:
X1_test_pred = pd.DataFrame(X1_test_pred)

In [148]:
X1_test_pred['ID'] = df2['ID']
X1_test_pred['genre'] = X1_test_pred[0]

In [149]:
X1_test_pred = X1_test_pred.drop(columns=[0])
X1_test_pred = X1_test_pred.set_index('ID')

In [150]:
X1_test_pred.to_csv('RF1_230116.csv',mode='w')